In [1]:
import sys
sys.path.append('..')

In [2]:
from dotenv import load_dotenv
from app.utils.data_tool import get_html_data_as_json, create_db_engine
import requests


assert load_dotenv()
engine = create_db_engine()
input_data = get_html_data_as_json(engine)
print('got the data')

/Users/alexander/Desktop/projects/haseigel-fs/.haseigel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


got the data


In [8]:
input_data['data'][0]['base_url'][0]

'https://podshipnik-mo.ru/stopornyye-shayby'

In [4]:
# Send POST request to localhost with input data
response = requests.post('http://127.0.0.1:5000/process-domains', json=input_data)

# Check if request was successful
if response.status_code == 200:
    print('Successfully sent data to server')
    print(response.json())
else:
    print(f'Error sending data: {response.status_code}')
    print(response.text)

Error sending data: 400
{
  "error": "unhashable type: 'dict'"
}



In [18]:
from app.services.image_models import MobileViTClassifier
from app.services.extract_images import download_images_with_local_path, extract_img_attributes
from collections import defaultdict
from app.config import TEMP_IMAGE_DIR

def process_single_domain(domain_data, model):
    domain_results = {
        "domain_start_id": domain_data["domain_start_id"],
        "predictions": [],
        "statistics": defaultdict(int)  # Will be populated based on actual predictions
    }
    
    # Process images
    for html in domain_data["response_text"]:
        img_data = extract_img_attributes(html)
        # print(img_data)
        # break
        for img in img_data:
            # First download the image and add local path
            download_images_with_local_path([img], TEMP_IMAGE_DIR)
            
            # Then check if download was successful and local path was added
            if "local_path" in img:
                # Skip SVG files since they can't be processed by the model
                if img["local_path"].lower().endswith('.svg'): ### TODO this is a wrong place to check it, we shouldnt download SVGs in the first place
                    continue
                prediction = model.predict(img["local_path"])['prediction']
                
                # Store individual prediction
                domain_results["predictions"].append({
                    "image_path": img["local_path"],
                    "predicted_class": prediction
                })
                
                # Update statistics counter
                domain_results["statistics"][prediction] += 1
    
    return domain_results


def process_domains(domains_data, output_type="detailed"):
    model = MobileViTClassifier()

    detailed_results = []
    summary_stats = {
        "total_domains": 0,
        "total_images": 0,
        "statistics": defaultdict(int)
    }
    
    for domain in domains_data["data"]:
        domain_results = process_single_domain(domain, model)
        detailed_results.append({
            "domain_start_id": domain["domain_start_id"],
            "statistics": dict(domain_results["statistics"]),  # Convert defaultdict to regular dict
            "predictions": domain_results["predictions"],
            "total_images": len(domain_results["predictions"])
        })
        
        # Update summary
        summary_stats["total_domains"] += 1
        summary_stats["total_images"] += len(domain_results["predictions"])
        for category, count in domain_results["statistics"].items():
            summary_stats["statistics"][category] += count

    # Convert summary_stats['statistics'] back to regular dict
    summary_stats["statistics"] = dict(summary_stats["statistics"])
    
    if output_type == "detailed":
        return {
            "status": "success",
            "output": {
                "details": detailed_results,
                "summary": dict(summary_stats)  # Convert defaultdict to regular dict
            }
        }
    else:
        return {
            "status": "success",
            "output": dict(summary_stats)  # Convert defaultdict to regular dict
        }

In [19]:
process_domains(input_data)

/Users/alexander/Desktop/projects/haseigel-fs/.haseigel/lib/python3.10/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


{'status': 'success',
 'output': {'details': [{'domain_start_id': 181,
    'statistics': {},
    'predictions': [],
    'total_images': 0},
   {'domain_start_id': 278,
    'statistics': {},
    'predictions': [],
    'total_images': 0},
   {'domain_start_id': 1187,
    'statistics': {},
    'predictions': [],
    'total_images': 0},
   {'domain_start_id': 1189,
    'statistics': {'prison, prison house': 1,
     'screen, CRT screen': 1,
     'swab, swob, mop': 27,
     'chime, bell, gong': 1,
     'hook, claw': 2},
    'predictions': [{'image_path': '/Users/alexander/Desktop/projects/haseigel-fs/data/images/temp/0_logo_kalydon.png',
      'predicted_class': 'prison, prison house'},
     {'image_path': '/Users/alexander/Desktop/projects/haseigel-fs/data/images/temp/0_iso_kalydon.png',
      'predicted_class': 'screen, CRT screen'},
     {'image_path': '/Users/alexander/Desktop/projects/haseigel-fs/data/images/temp/0_WEB-RUBI.jpg',
      'predicted_class': 'swab, swob, mop'},
     {'image

In [20]:
ouputs = _

In [21]:
ouputs['output']['summary']

{'total_domains': 9,
 'total_images': 51,
 'statistics': {'prison, prison house': 1,
  'screen, CRT screen': 1,
  'swab, swob, mop': 27,
  'chime, bell, gong': 1,
  'hook, claw': 3,
  'slot, one-armed bandit': 2,
  'harmonica, mouth organ, harp, mouth harp': 1,
  'analog clock': 4,
  'digital clock': 1,
  'scoreboard': 1,
  'book jacket, dust cover, dust jacket, dust wrapper': 6,
  'chain': 1,
  'safety pin': 1,
  'oboe, hautboy, hautbois': 1}}